In [1]:
import argparse
import csv
import operator
import codecs
import cStringIO
import urllib2
import nltk
import sys
import movie_scrapers

# STEP 1: Generate List of Animated Movies Released Since 1980

In [2]:
test_wiki_scraper = movie_scrapers.scrape_movie_lists('https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_2016')
print '\n'.join(test_wiki_scraper[0:10])

Accel World: Infinite Burst
Ajin -Shōtotsu-
Ajin: Shōgeki
Albert
The Angry Birds Movie
Another Day of Life
The Ape Story
Ballerina
Big Fish & Begonia
BoBoiBoy: The Movie


In [14]:
urls = []
BASE_URL = "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_"
year = 1980
while year < 2017:
    urls.append(BASE_URL+"{}".format(year))
    year += 1

In [15]:
# Call the function scrape_movie_lists for each year, starting with 1980:

movie_titles_all = []
for url in urls:
    movie_titles_all.append(movie_scrapers.scrape_movie_lists(url))
movie_titles_all = reduce(operator.concat, movie_titles_all)
print len(movie_titles_all)

2632


In [16]:
print '\n'.join(movie_titles_all[0:10])

15 Sonyeon Uju Pyoryugi
Animalympics
Be Forever Yamato
Bloody Lady
Bon Voyage, Charlie Brown (and Don't Come Back!!)
Cyborg 009: Legend of the Super Galaxy
Doksuri 5 Hyeongje
Doraemon: The Motion Picture
Eleven Hungry Cats
Fumoon


In [17]:
# Copied from python documentation:

class UnicodeWriter:
    """
    A CSV writer which will write rows to CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        # Redirect output to a queue
        self.queue = cStringIO.StringIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode("utf-8") for s in row])
        # Fetch UTF-8 output from the queue ...
        data = self.queue.getvalue()
        data = data.decode("utf-8")
        # ... and reencode it into the target encoding
        data = self.encoder.encode(data)
        # write to the target stream
        self.stream.write(data)
        # empty queue
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
        

In [348]:
# Save the list of animated movies in a csv file:

with open('movie_list.csv', 'wb') as output:
    writer = UnicodeWriter(output, lineterminator='\n', quoting=csv.QUOTE_NONNUMERIC)
    for val in movie_titles_all:
        writer.writerow([val])  

# STEP 2: Scrape IMDB for Full Cast and Crew, Year of Release, and Rating

 The hyperlink I want looks like: http://www.imdb.com/title/tt2096673/fullcredits?ref_=tt_cl_sm#cast

In [146]:
# Call the function get_imdb_links for each movie: 

imdb_urls = []
current_idx = 0
m = len(movie_titles_all)
for movie in movie_titles_all:
    imdb_urls.append(movie_scrapers.get_imdb_links(movie))
    
    if (current_idx % 10) == 0:
        progress = 100.0 * current_idx / m
        sys.stdout.write('\r progress {} / {} ({:6.2f}%)'.format(current_idx, m, progress))
        sys.stdout.flush()
    current_idx += 1
        
print len(imdb_urls)

 progress 2630 / 2633 ( 99.89%)2633


In [150]:
# Save the list of imdb movie links in a csv file:

with open('imdb_url_list.csv', 'wb') as output:
    writer = UnicodeWriter(output, lineterminator='\n', quoting=csv.QUOTE_NONNUMERIC)
    for val in imdb_urls:
        if val:
            val = val[0:17]
            writer.writerow([val])  

Note that the length of the CSV file is only 1922 rows. This indicates that ~27% of the movies released and published to wikipedia were not on IMDB. I spot checked a few movies, and most of the missing 27% were foreign animated movies.

In [18]:
# Test the Movie class on a movie with missing data:

random_movie = movie_scrapers.Movie('/title/tt4518910/')
print random_movie.rating
print random_movie.get_team_members("Writing Credits")

None
[u' Fais Umer\n']


### Note: json file containing data for all 1922 movies was generated outside of jupyter notebook